In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from re import sub, search, findall
import os
import pandas as pd
import numpy as np
from glob import glob
def remover(file_):
    file_ = ''.join(file_.split()).lower()
    stop_element = ['менее|<|>|автоматическая|ручная|[()]'
              '|меньш.|больше|более|до(?=[0-9])|менее|iultra|от(?=[0-9])'] #'до' ищет только после чисел, тк идет захват 'биохимисслеДОваниекрови'. аспартатаминОТрансфераза = аспартатаминрансфераза
    for elem in stop_element:
      file_ = sub(elem, '', file_)
    file_ = sub(r',', '.', file_) # иначе не меняет на точку 
    return(file_)
def change_date_format(dt):
    """ на первом этапе переводим "нормальный формат" в формат через дефисы
     на втором этапе ищем вариант дд.мм.гг в формат через дефисы"""
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', dt)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})', 
                  '\\1-\\2-20\\3', data_1)

In [6]:
lst = list(glob(os.path.join('/content/drive/MyDrive/parser', '*')))

In [11]:
test_dates = []
patients_id = []
patients_birthday = []
values = []
values_cre, values_ast, values_alt, values_ldh = [], [], [], []
for doc in lst:
  with open(doc, encoding='utf-8', errors='ignore') as file_:
#with open('/content/drive/MyDrive/проверка/германович-6.txt') as file_:
    file_ = change_date_format(remover(file_.read()))
    file_1 = sub(r'(биохимическоеисследованиекрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) 
    for value in findall(r'(\d{2}-\d{2}-\d{4}биохимическоеисследованиекрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1):
      date = search(r'(\d{2}-\d{2}-\d{4})(?=биохимическоеисследованиекрови)', file_1)
      if date:
        test_dates.append(date[0])
      id = search(r'\d+', file_1) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
      if id:
        patients_id.append(id[0])
      birthday = search("(\d{2})-(\d{2})-(\d{4})", file_1)
      if birthday:
        patients_birthday.append(birthday[0])
      crp = search(r'(?<=.-реактивныйбелок)\d+', value)
      if crp:
        values.append(crp[0])
      else:
        values.append('NaN')
      value_cre = search(r'(?<=.р.ат.нин)\d+', value)
      if value_cre:
        values_cre.append(value_cre[0])
      else:
        values_cre.append('NaN') 
      ast = search(r'(?<=.сп.ртат.м.н.тр.нсф.раз.)\d+', value)            
      if ast:
        values_ast.append(ast[0])
      else:
        values_ast.append('NaN')
      alt = search(r'(?<=.л.н.н.м.н.тр.нсф.раз.)\d+', value)
      if alt:
        values_alt.append(alt[0])
      else:
        values_alt.append('NaN')
      ldh = search(r'(?<=л.кт.тд.г.др.г.наз.)\d+', value)
      if ldh:
        values_ldh.append(ldh[0])
      else:
        values_ldh.append('NaN')

In [12]:
data_dct = {'ID':patients_id, 'Birthday':patients_birthday, 'dates':test_dates, 'values_CRP':values, 'values_cre':values_cre, 'values_ast':values_ast, 'values_alt':values_alt, 'values_ldh':values_ldh}
df = pd.DataFrame(data_dct)
df = df.replace(r'', np.nan, regex=True)
df

,ID,Birthday,dates,values_CRP,values_cre,values_ast,values_alt,values_ldh
0,1025,19-01-2000,14-02-2020,6,NaN,21,15,NaN
1,7658,24-08-1993,06-12-2018,36,NaN,NaN,NaN,NaN
2,7658,24-08-1993,06-12-2018,6,NaN,NaN,NaN,NaN
3,3418,17-06-1941,23-05-2018,24,84,30,16,NaN
4,3418,17-06-1941,23-05-2018,12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2542,5590,03-02-1942,07-09-2018,12,NaN,NaN,NaN,NaN
2543,2676,29-06-1981,11-06-2020,2,51,21,14,NaN
2544,2676,29-06-1981,11-06-2020,3,NaN,63,77,NaN
2545,3986,20-04-1967,19-06-2018,6,93,22,18,NaN
